### Background

## 1. Set up drive

In [1]:
# Mount Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
cd "gdrive/MyDrive/Projects/1 - Numericals/Autism Prediction Challenge/2 - Production/data"

/content/gdrive/MyDrive/Projects/1 - Numericals/Autism Prediction Challenge/2 - Production/data


## 2. Import Libraries

In [3]:
# Load data
import pandas as pd
import numpy as np
import io
import os
import glob

# Meta
import time

# Visualizations
import matplotlib.pyplot as plt
import seaborn as sb

# Analysis
from scipy.stats import zscore

# Label Encoder
from sklearn import preprocessing

## 3. Data Assessment Class

## 3. Load Data

In [4]:
ls

Autism-prediction/     l1_clean_df.csv        test.csv
autism-prediction.zip  sample_submission.csv  train.csv


In [5]:
df = pd.read_csv('train.csv')

## 4. Clean Data

1. Age - It has at least 6 decimal points - Reduce it to 1 decimal point


Code:

In [6]:
df['age'] = df['age'].round(2) 

Test:

In [7]:
df['age']

0      18.61
1      13.83
2      14.68
3      61.04
4      14.26
       ...  
795    42.08
796    17.67
797    18.24
798    19.24
799    32.17
Name: age, Length: 800, dtype: float64

2. Result - The result column is of float type - Ensure result is rounded to 6 decimal places

Code:

In [8]:
df['result'] = df['result'].round(6) 

Test:

In [9]:
df['result']

0       7.819715
1      10.544296
2      13.167506
3       1.530098
4       7.949723
         ...    
795    13.390868
796     9.454201
797     6.805509
798     3.682732
799    12.060168
Name: result, Length: 800, dtype: float64

3 - Impute misisng values in ethnicity and relation features

Code:

In [10]:
# Replace ? with the word 'Unknown' in the dataframe

df = df.replace(to_replace = '?', value = 'Unknown')

Test:

In [11]:
df[df['ethnicity'] == '?']

,ID,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,...,gender,ethnicity,jaundice,austim,contry_of_res,used_app_before,result,age_desc,relation,Class/ASD


In [12]:
df[df['relation'] == '?']

,ID,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,...,gender,ethnicity,jaundice,austim,contry_of_res,used_app_before,result,age_desc,relation,Class/ASD


4 - The 'others' value in Ethnicity is in lower case. This needs to be changed to title case

Code:

In [13]:
df['ethnicity'] = df['ethnicity'].str.title()

Test:

In [14]:
df['ethnicity'].str.istitle().unique()

array([ True])

5 - Change column names

Code:

In [15]:
# The ‘austim’ column is not descriptive and there is a typo error - Change it to ‘fam_history_autism’
df = df.rename(columns={"austim":"fam_history_autism"})

In [16]:
# The ‘contry_of_res’ column is not descriptive and there is a typo error - Change it to ‘Country’
df = df.rename(columns={"contry_of_res":"country"})

In [17]:
# Non descriptive column header result  - Change it to ‘final_score’
df = df.rename(columns={"result":"final_score"})

In [18]:
# Non descriptive column header relation  - Change it to ‘survey_done_by’
df = df.rename(columns={"relation":"survey_done_by"})

Test:

In [19]:
df.columns

Index(['ID', 'A1_Score', 'A2_Score', 'A3_Score', 'A4_Score', 'A5_Score',
       'A6_Score', 'A7_Score', 'A8_Score', 'A9_Score', 'A10_Score', 'age',
       'gender', 'ethnicity', 'jaundice', 'fam_history_autism', 'country',
       'used_app_before', 'final_score', 'age_desc', 'survey_done_by',
       'Class/ASD'],
      dtype='object')

6 - The AQ1-10 screening test recommended for adults without moderate or severe learning disability. Hence this model is useful only for adults. Also, the participants in the dataset are also adults - Drop age_desc feature

Code:

In [20]:
df = df.drop(['age_desc'], axis = 1)

Test:

In [21]:
df.columns

Index(['ID', 'A1_Score', 'A2_Score', 'A3_Score', 'A4_Score', 'A5_Score',
       'A6_Score', 'A7_Score', 'A8_Score', 'A9_Score', 'A10_Score', 'age',
       'gender', 'ethnicity', 'jaundice', 'fam_history_autism', 'country',
       'used_app_before', 'final_score', 'survey_done_by', 'Class/ASD'],
      dtype='object')

7 - Save data to csv

In [22]:
df.to_csv('l1_clean_unencoded.csv')

8 - Encode categorical data

Code:

In [24]:
def get_quantitative_lists(df):
    '''
    Function: Get the categorical column names of a dataframe 
    Input: A dataframe
    Output: The categorical and numerical column name lists
    '''

    numeric_columns = df.select_dtypes([np.number]).columns.tolist()
    columns = list(df)
    categorical_columns = []
    for i in columns:
      if i not in numeric_columns:
        categorical_columns.append(i)
      else:
        None
    
    return categorical_columns

In [25]:
# Get categorical columns as a list
category_columns = get_quantitative_lists(df)

In [27]:
# Apply label encoding to categorical columns
df[category_columns] = df[category_columns].apply(preprocessing.LabelEncoder().fit_transform)

Test:

In [28]:
df

,ID,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,...,age,gender,ethnicity,jaundice,fam_history_autism,country,used_app_before,final_score,survey_done_by,Class/ASD
0,1,1,0,1,1,1,1,0,1,1,...,18.61,0,10,0,0,58,0,7.819715,4,0
1,2,0,0,0,0,0,0,0,0,0,...,13.83,0,7,0,0,6,0,10.544296,5,0
2,3,1,1,1,1,1,1,0,0,1,...,14.68,0,10,0,0,57,0,13.167506,4,1
3,4,0,0,0,1,0,0,0,0,0,...,61.04,0,7,0,0,39,0,1.530098,5,0
4,5,0,0,0,0,1,0,0,0,1,...,14.26,1,1,0,1,32,0,7.949723,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,796,1,1,1,1,1,1,1,1,1,...,42.08,0,10,0,1,58,0,13.390868,4,1
796,797,1,1,0,0,1,0,0,0,1,...,17.67,0,0,0,0,39,0,9.454201,4,0
797,798,0,0,0,0,0,0,1,0,1,...,18.24,1,10,1,0,34,0,6.805509,4,1
798,799,1,1,1,1,1,1,0,1,1,...,19.24,0,4,0,1,58,0,3.682732,3,0


9 - Save encoded data

In [29]:
df.to_csv('l1_clean_encoded.csv')